# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [19]:
def get_gini(df, label):
    gini = 1 #초기값 1로 지정
    total = len(df[label]) #전체 행 개수
    number = df[label].value_counts() # 타겟변수 class별 개수 파악
    for i in number: # class별 데이터 개수/전체 행 개수를 제곱한 값을 gini에서 빼줌(클래스 개수 만큼 반복될것)
        gini -= (i/total)**2
    return gini

In [20]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [23]:
import itertools
from itertools import combinations # 변수의 모든 클래스 조합을 얻기 위해 itertools 불러오기
def get_binary_split(df, attribute):
    x = list(df[attribute].unique()) #변수의 class들 목록 리스트에 저장
    result = [] #결과담을 빈 리스트 생성
    for i in range(1,len(x)): #class종류 개수로 for문 생성(처음 0값은 아무것도 안뽑는 경우라 1부터 시작)
        a = list(combinations(x,i)) #itertools의 조합 함수 combination을 이용하여 결과를 리스트에 저장
        for j in range(len(a)): #combinaiton결과 튜플형태로 나오기 때문에 for문을 활용하여 원소 하나씩 리스트로 바꿔주는 작업
            b = list(a[j])
            result.append(b)
    
    
    return result

In [24]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [25]:
def get_attribute_gini_index(df, attribute, label):
    split = get_binary_split(df, attribute) #이진분류 리스트부터 뽑아놓고
    result ={} # 빈 dictionary 생성
    for i in split: #이진분류 리스트속 리스트들을 하나씩 뜯어보자
        name = list(df[attribute].unique()) #해당하는 변수 class들을 리스트로 뽑아놓고
        for j in range(len(i)): #리스트의 원소들을 하나씩 보자
            if j == 0: #첫번째 원소를 값으로 갖는 데이터들만 new라는 df에 집어넣고
                new = df.loc[df[attribute] == i[j]]
            else: #(리스트 안에 다른 원소들이 존재한다면)나머지 원소들에 해당하는 데이터만 뽑아서 new라는 데이터프레임에 추가하기
                new = pd.concat([new,df.loc[df[attribute] == i[j]]],axis=0)
        #새로만든new의 행 개수/전체df  행 개수 * 각 라벨의 지니계수
        kio = len(new[attribute])/len(df[attribute]) * get_gini(new,label) 
        
        # 이진분류 리스트에 들어있지 않은 원소들에 대해서도 지니계수를 구해야하기 때문에
        name = list(df[attribute].unique()) #해당 변수의 class를 리스트로 저장해두고 
        for k in i: #i 리스트에 속한 원소들 하나씩 제거
            name.remove(k)
          
        for s in range(len(name)): #나머지는 위의 지니계수 구하는 방법과 동일
            if s == 0:
                new_2 = df.loc[df[attribute] == name[s]]
            else:
                new_2 = pd.concat([new_2,df.loc[df[attribute] == name[s]]],axis=0)
        kio_2 = len(new_2[attribute])/len(df[attribute]) * get_gini(new_2,label)
                
        gini = kio + kio_2 #두 값 더해주기
        
        result[tuple(i)] = gini  #하나씩 딕셔너리에 추가하기
        
    
    return result

In [26]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [35]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [13]:
## 문제1) 변수 income의 이진분류 결과를 보여주세요.
get_binary_split(pd_data, "income")


[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [15]:
## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

# 1. age
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 2. income
my_dict = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 3. student
my_dict = get_attribute_gini_index(pd_data, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 4. credit_rating
my_dict = get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715
Min - ('high',) : 0.4428571428571429
Min - ('no',) : 0.3673469387755103
Min - ('fair',) : 0.42857142857142855


가장 작은 값은 age 변수의 0.35714285714285715

In [16]:
#age 변수의 gini index
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

In [40]:
## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

# age 변수로 데이터 프레임 split(new_data1, new_data2)
new_data1 = pd_data.loc[pd_data['age']== 'middle_aged']
new_data2_1 = pd_data.loc[pd_data['age']== 'senior']
new_data2_2 = pd_data.loc[pd_data['age']== 'youth']
new_data2 = pd.concat([new_data2_1,new_data2_2],axis=0)

In [37]:
new_data1

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [41]:
new_data2

,age,income,student,credit_rating,class_buys_computer
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
9,senior,medium,yes,fair,yes
13,senior,medium,no,excellent,no
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes


In [43]:
##1. new_data2를 기준으로 그 다음으로 중요한 변수를 찾아보자

# 2. income
my_dict = get_attribute_gini_index(new_data2, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 3. student
my_dict = get_attribute_gini_index(new_data2, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

# 4. credit_rating
my_dict = get_attribute_gini_index(new_data2, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('high',) : 0.375
Min - ('no',) : 0.31999999999999984
Min - ('fair',) : 0.4166666666666667


최솟값은 student변수의 0.31999999999999984

In [44]:
#student 변수의 gini_index
get_attribute_gini_index(new_data2, "student", "class_buys_computer")

{('no',): 0.31999999999999984, ('yes',): 0.31999999999999984}